<a href="https://colab.research.google.com/github/minjiekim/geotweets/blob/main/geosearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [ ]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [ ]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets_2.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "api key"
    consumer_secret = "api key secret"
    access_token = "access token"
    access_token_secret = "access token secret"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-87.960805, 24.694937, -64.038592, 47.760006] 

    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1517683850914246656, 'username': 'SyneArdwin', 'created_at': 'Sat Apr 23 01:56:41 +0000 2022', 'lng': -87.74796549999999, 'lat': 43.7485585, 'text': '@Allikuja Happy birthday!'}
{'id': 1517683851224621057, 'username': 'asuabransito', 'created_at': 'Sat Apr 23 01:56:41 +0000 2022', 'lng': -79.4997615, 'lat': 43.617268499999994, 'text': 'See you soon Montreal…'}
{'id': 1517683852575137796, 'username': '511NY', 'created_at': 'Sat Apr 23 01:56:42 +0000 2022', 'lng': -78.767543, 'lat': 42.893079, 'text': 'Updated: Incident on #I90NYSThruway EB at Before Exit 52A (I-90) - Buffalo - William St https://t.co/Wu7OtKKJh9'}
{'id': 1517683853531430912, 'username': '511nyWNY', 'created_at': 'Sat Apr 23 01:56:42 +0000 2022', 'lng': -78.767543, 'lat': 42.893079, 'text': 'Updated: Incident on #I90NYSThruway EB at Before Exit 52A (I-90) - Buffalo - William St https://t.co/hZ3n2x3kAs'}
{'id': 1517683853258854400, 'username': 'SharonPinchback', 'created_at': 'Sat Apr 23 01:56:42 +0000 2022', 'lng':

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
